<a href="https://colab.research.google.com/github/azarenkovgd/consents-recognition/blob/master/superGetData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/iinti')

Mounted at /content/drive


In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install pymorphy2
!pip install pymystem3
!pip install tensorflow-gpu==1.15.2
!pip install pandas==1.0.0
!pip install jsonlines
clear_output()

In [ ]:
!pip install jsonlines

In [ ]:
import os
import re
import gc
import json
import nltk
import pymorphy2
import numpy as np
import pandas as pd 

from pymystem3 import Mystem
from nltk.corpus import words
from nltk.corpus import stopwords

In [ ]:
import json
import jsonlines
import pandas as pd
import numpy as np

#GetData 

*   (Просто решил сделать в одном файле для удобства)




In [ ]:
!pip install jsonlines
clear_output()

In [ ]:
def load_json(path: str):
    with open(path, 'r', encoding='utf-8') as read_file:
        data = json.load(read_file)
    return data

In [ ]:
with open('data/test.jsonl', 'r') as json_file:
        json_list = list(json_file)

In [ ]:
def get_X_y(data_json_file):
    X, y = [], []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    X.append(text + query + a['text'])
                    y.append(a['label'])
    return X, y

In [ ]:
X_train, y_train = get_X_y('data/train.jsonl')
X_test, y_test = get_X_y('data/val.jsonl')

In [ ]:
train_df = pd.DataFrame({
    'text': X_train,
    'labels':y_train
})

eval_df = pd.DataFrame({
    'text': X_test,
    'labels': y_test
})



# Функции

In [ ]:
def save_csv(df,path_out, sep=',', encoding='utf-8', index=False, header=True):
    #Сохраняем CSV-файл
    path_out = path_out.replace('\\','/')

    if not os.path.exists('/'.join(path_out.split('/')[0:-1])):
        os.makedirs('/'.join(path_out.split('/')[0:-1]))
        
    df.to_csv(path_out, index=index, header=header, sep=sep, encoding=encoding)

In [ ]:
def fillna(df, column_str, replace_to=''):
    #Заполнение пропусков

    df[column_str] = df[column_str].fillna(replace_to)

In [ ]:
def clear_chars(text):
    #Удаляем лишние символы, включая лишние пробелы и переносы строк

    text = re.sub('[^a-zA-Zа-яА-Я 0-9]+', ' ', text) 
    return ' '.join(text.split())

In [ ]:
def do_lower(text):
    #Приводим текст к нижнему регистру

    return text.lower()

In [ ]:
def creat_stop_words(df, text_str, label_str, stop_words_excep, stop_words_extra, min_labels_count=None):
    #Формируем список стоп-слов (слова которые встречаются в заданном числе меток)
    # min_labels_count - минимальное кол-во категорий в которых должно содержаться стоп-слово

    # Список меток
    labels = list(set(df[label_str].to_list()))
    
    # Кол-во категорий в которых должно содержаться стоп-слово
    if min_labels_count is None:
        min_labels_count = len(labels)

    #Список слов для каждой метки
    labels_word = {}
    for label in labels:
        labels_word[label]= get_dict(df[df[label_str] == label],text_str)
        
    # Формируем список стоп-слов  
    stop_words = []
    uniq_words = get_dict(df,text_str)
        
    for word in uniq_words:
        count = 0
        for label in labels:
            if word in labels_word[label]:
                count+=1
        if count >= min_labels_count and word not in stop_words_excep:
            stop_words.append(word)
    
    stop_words.extend(stop_words_extra)
    return stop_words 

In [ ]:
def delete_stopwords(text):
    # Очистка от стоп-слов
    # stops - список стоп слов. Если пустой то используем стандартные

    global stop_words
    if stop_words is None:
        stop_words = set(stopwords.words("english")) | set(stopwords.words("russian"))

    new_sentence = [word for word in text.split() if not word.lower() in stop_words]
    return ' '.join(new_sentence)

In [ ]:
def pymorphy_lemmatization_ru(text,fix_case=False,fix_aggressive=False):
# Лемматизатор для русских текстов (pymorphy)

        new_sentence = [morph_ru.parse(word)[0].normal_form for word in text.split()]
        if fix_case:
            fix_sentence = []
            old_sentence = text.split()
            for old,new in zip(old_sentence,new_sentence):
              fix_sentence.append(fix_text_case(old,new,fix_aggressive))
            return ' '.join(fix_sentence) 
        return ' '.join(new_sentence)

In [ ]:
def pymystem_lemmatization_ru(text,fix_case=False,fix_aggressive=False):
    # Лемматизатор для русских текстов (pymystem)

        new_sentence = ''.join(lemmatizator.lemmatize(text)[:-1]).split()
        if fix_case:
            fix_sentence = []
            old_sentence = text.split()
            for old,new in zip(old_sentence,new_sentence):
              fix_sentence.append(fix_text_case(old,new,fix_aggressive))
            return ' '.join(fix_sentence) 
        return ' '.join(new_sentence)

In [ ]:
def do_lemmatization(text, fix_case=True, fix_aggressive=False):
    # Лемматизация текста

    if lemm_by == 'pymorphy':
        text = pymorphy_lemmatization_ru(text,fix_case=fix_case,fix_aggressive=False)
    else:
        text = pymystem_lemmatization_ru(text,fix_case=fix_case,fix_aggressive=False)
    return text 

In [ ]:
def load_spellFix_model(vocab_path=None,lang_model_path=None,download=False):
    # Загружаем предобученную модель исправления опечаток

    spell_model_config = json.load(open(configs.spelling_correction.levenshtein_corrector_ru))
    
    if vocab_path:
        spell_model_config['chainer']['pipe'][2]['load_path'] = vocab_path
        spell_model_config['chainer']['pipe'][2]['save_path'] = vocab_path

    if lang_model_path:
        spell_model_config['chainer']['pipe'][4]['load_path'] = lang_model_path
        
    return build_model(spell_model_config, download=download)

In [ ]:
def fixSpell_ru(text,fix_case=True,fix_aggressive=False):
    # Исправление опечаток в русских текстах

    new_sentence = model_spell([text])[0]    
    if fix_case:
            return fix_text_case(text,new_sentence,fix_aggressive)
    return new_sentence    

In [ ]:
def fix_keyboard_layout(word):
    # Проверяем слово на корректность раскладки клавиатуры при его наборе
    
    def check_layot(word):
        ru = 0
        en = 0
        for char in word:
            if char in alphabet_ru:
                ru+=1
            if char in alphabet_en:
                en+=1
        return 'ru' if ru >= en else 'en'
    
    def switch_layot(word,to_layot):
        new_word = ''
        for char in word:
            was_find = False
            for ru,en in zip(layout_ru,layout_en):
                if to_layot == 'en':
                    if char == ru:
                        new_word+= en
                        was_find = True
                        break
                else:
                    if char == en:
                        new_word+= ru
                        was_find = True
                        break 
            if not was_find:
                new_word+= char
        return new_word                       
        
        
    current_layot = check_layot(word)
    if current_layot == 'ru':
        russian = word_is_ru(word)
    else:
        english = word_is_en(word)
        
    if current_layot == 'ru' and russian:  
        return word
    elif current_layot == 'ru' and not russian:
        word_en = switch_layot(word,'en')
        english = word_is_en(word_en)
        return word_en if english else word
    elif current_layot == 'en' and english:
            return word
    elif current_layot == 'en' and not english:
        word_ru = switch_layot(word,'ru')
        russian = word_is_ru(word_ru)
        return word_ru if russian else word    

In [ ]:
def tokenize_by_rules(string):
# Токенизируем строку по заданной группе(правилам разделения)

        token = ''
        tokens = []
        category = None
        categories = ['0123456789','абвгдеёжзийклмнопрстуфхцчшщъыьэюя','abcdefghijklmnopqrstuvwxyz']
        for char in string:
            if token:
                if category and char.lower() in category:
                     token += char
                else:
                     if not token == ' ':
                         tokens.append(token)
                     token = char
                     category = None
                     for cat in categories:
                         if char.lower() in cat:
                             category = cat
                             break
            else:
                 category = None
                 if not category:
                     for cat in categories:
                         if char.lower() in cat:
                             category = cat
                             break
                 token += char
        if token:
             if not token == ' ':
                 tokens.append(token)
        return ' '.join(tokens)

In [ ]:
def replace_empty_to_max_freq_label(df,text_str,label_str,empty_str=''):
# Определяем наиболее частую метку для пропущенных данных
# и присваиваем её для всех попусков.

        df_empty = df[df[text_str] == empty_str]
        df_empty_uniq = pd.value_counts(df_empty.values.ravel())[1:]
        try:
            max_freq_label = df_empty_uniq.index[0]

            df_empty_indx = df_empty.index.to_list()
            for empty_indx in df_empty_indx:
                df.loc[empty_indx,label_str] = max_freq_label
        except:
            pass

In [ ]:
def word_is_en(word):
        return word.lower() in vocab_en

In [ ]:
def word_is_ru(word):
        return morph_ru.word_is_known(word.lower(), strict_ee=False)

In [ ]:
def get_list_of_unknow_words(words_list):
        """ Список неизвестных слов
        """
        unknow = []
        for word in words_list:
            if not word_is_ru(word) and not word_is_en(word):
                unknow.append(word)
        return unknow

In [ ]:
def get_dict(df,column_str='',uniq=True):
        """ Список уникальных слов в конкретном столбце
        """
        if not isinstance(df,list):
            rows = df[column_str].to_list()
        else:
            rows = df
        dictionary = []
        for row in rows:
            dictionary.extend(row.split())
        return list(set(dictionary)) if uniq else dictionary

In [ ]:
def save_Dataset(dataset,folder_path,sep=',',encoding='utf-8',index=False,header=True,postfix=''):
        """ Сохраняем датасет
        """  
        if 'train' in dataset:
            save_csv(dataset['train'], os.path.join(folder_path, "train" + postfix + ".csv"),
                    sep=sep,encoding=encoding,index=index,header=header)
        if 'test' in dataset:
            save_csv(dataset['test'], os.path.join(folder_path, "test" + postfix + ".csv"),
                    sep=sep,encoding=encoding,index=index,header=header)
        if 'valid' in dataset:
            save_csv(dataset['valid'], os.path.join(folder_path, "valid" + postfix + ".csv"),
                    sep=sep,encoding=encoding,index=index,header=header)  

#Подготовка данных

### Провести нормальную аналитику (задача)

In [ ]:
stop_words = None
stop_words_excep = ['интернет','оператор','тариф','интернета','работает','нет']
stop_words_extra = ['добрый','доброго','добры','доброй','доброе','добрые',
                    'здравствуйте','здрасте','здравчтвуйтет','здрастивити',
                    'здравствуй','здрастувуйте','здраствутйте','здрсти','здрьте',
                    'вопросздравствуйте','введитездравствуйте','здрасьте','здрастуйте',
                    'здраастауйте','введздравствуйте','здравсте','привет','приветствую'
                    'или','который','очень','еще','это']

alphabet_en = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
alphabet_ru = ['а','б','в','г','д','е','ё','ж','з','и','й','к','л','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ъ','ы','ь','э','ю','я']
layout_en = ['q','w','e','r','t','y','u','i','o','p','[',']','a','s','d','f','g','h','j','k','l',';','','z','x','c','v','b','n','m',',','.','`']
layout_ru = ['й','ц','у','к','е','н','г','ш','щ','з','х','ъ','ф','ы','в','а','п','р','о','л','д','ж','э','я','ч','с','м','и','т','ь','б','ю','ё']

# Словарь английских слов (для проверки на корректность раскладки)
nltk.download('words')
vocab_en = set(w.lower() for w in words.words())

model_spell = None

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
# Экземпляр класса pymorphy2
morph_ru = pymorphy2.MorphAnalyzer()

# Экземпляр класса pymystem3 (в разы медленнее)
lemmatizator = Mystem()

In [ ]:
# Параметры предобработки
do_clen = True
do_lower_case = False # оставляем регистр без изменений т.к. будем использовать BERT модель обученную на данных без перевода в нижний регистр
replace_empty_label = True
do_tokenize = True
do_fix_spell = False
do_lemm = True
lemm_by = 'pymorphy' # pymorphy, pymystem
del_stops = True
drop_dupl = True
fix_case = True
fix_layout = True

## Принты

In [ ]:
# Заменяем пропуски
print('Заменяем пропуски..')
fillna(train_df,'text',replace_to='')
fillna(eval_df,'text',replace_to='')
    
# Удаляем лишние символы
if do_clen:
    print('Удаляем лишние символы..')
    train_df['text'] = train_df['text'].apply(clear_chars)
    eval_df['text'] = eval_df['text'].apply(clear_chars)

    
# Переводим в нижний регистр
if do_lower_case:
    print('Переводим в нижний регист..')
    train_df['text'] = train_df['text'].apply(do_lower)
    eval_df['text'] = eval_df['text'].apply(do_lower)


# Определяем наиболее частую метку для пропусков и присваиваем её для всех пропусков в тренировочных данных
if replace_empty_label:
    print('Заменяем пустые метки..')
    replace_empty_to_max_freq_label(train_df,text_str='text',label_str='label',empty_str='')
    replace_empty_to_max_freq_label(eval_df,text_str='text',label_str='label',empty_str='')
    
# Токенизируем текст (отделяем числа от текста и английский текст от русского)
if do_tokenize:
    print('Токенизация..')
    train_df['text'] = train_df['text'].apply(tokenize_by_rules)
    eval_df['text'] = eval_df['text'].apply(tokenize_by_rules)


# Формируем список стоп-слов (слова которые встречаются в заданном числе меток)
if del_stops:
    print('Формируем список стоп-слов..')
    stop_words = creat_stop_words(train_df,'text','labels',
                                  stop_words_excep,
                                  stop_words_extra,
                                  min_labels_count=None)
    stop_words = creat_stop_words(eval_df,'text','labels',
                                  stop_words_excep,
                                  stop_words_extra,
                                  min_labels_count=None)


    # Удаляем стоп-слова
    print('Удаляем стоп-слова..')
    train_df['text'] = train_df['text'].apply(delete_stopwords)
    eval_df['text'] = eval_df['text'].apply(delete_stopwords)

# Заменяем пустые строки на пробел (фикс ошибки при обучении BERT)
print('Заменяем пустые строки на пробел..')
train_df['text'] = train_df['text'].replace('',' ')
eval_df['text'] = eval_df['text'].replace('',' ')


Заменяем пропуски..
Удаляем лишние символы..
Заменяем пустые метки..
Токенизация..
Формируем список стоп-слов..
Удаляем стоп-слова..
Заменяем пустые строки на пробел..


# Сохранение датасета

In [ ]:
train = train_df.to_csv('data/train.csv')
eval = eval_df.to_csv('data/eval.csv')